In [24]:
%matplotlib inline
from __future__ import print_function
import os,sys
notebook_code_root = os.path.dirname(os.getcwd())
data_root = os.path.dirname(notebook_code_root) + '/data/'
print("Data root: %s" % data_root)
sys.path.insert(0,notebook_code_root)

import bcolz
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
import math
import numpy as np
from vgg16 import *

from IPython.display import FileLink

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
    
def load_array(fname):
    return bcolz.open(fname)[:]


class NotebookData:
    def __init__(self,
                 data_dir,
                 data='statefarm',
                 results_dir='results',
                 sample_mode=True,
                 train=True,
                 preprocess=True):
        
        self.data_root = data_dir + data + '/'
        self.sample_root = self.data_root + 'sample/'
        self.sample_mode = sample_mode
        self.sample_batch_size = 1
        self.rdir = results_dir
        self.train = train
        self.preprocess = preprocess
        self.training_data = None
        self.validation_data = None
        self.training_labels = None
        self.validation_labels = None
        
    def root_dir(self):
        return self.sample_root if self.sample_mode else self.data_root
    
    def test_dir(self):
        return self.root_dir() + 'test/'
    
    def results_dir(self):
        return self.root_dir() + self.rdir
    
    def train_dir(self):
        return self.root_dir() + 'train/'

    def valid_dir(self):
        return self.root_dir() + 'valid/'
    
    def pproc_dir(self):
        return self.root_dir() + 'preprocessed/'
    
    def batch_size(self, requested_size=8):
        return self.sample_batch_size if self.sample_mode else requested_size
    
    def load_data_and_labels(self):
        """Loads the batches and labels to the internal state.
        
           Upon loading, the data is accessible through the
           corresponding methods.
        """
        gen = image.ImageDataGenerator()
        target_size = (224,224)
        if self.preprocess:
            print('Preprocessing data...')
            if not os.path.isdir(self.pproc_dir()):
                os.mkdir(self.pproc_dir())
                
            batch_arr = []
            for ld,segment in [(self.train_dir(), 'train'),
                          (self.valid_dir(), 'valid')]:
                # TODO(ness): segment = os.basename(ld)
                flowgen = gen.flow_from_directory(
                    ld,
                    target_size=target_size,
                    shuffle=False,
                    class_mode=None,
                    batch_size=1)
                # Save the batches using method defined in utils.py
                data = np.concatenate([flowgen.next() for i in range(flowgen.n)])
                batches_dir = self.pproc_dir() + segment + '-bc'
                save_array(batches_dir, data)
                
                # Save the classes.
                cls_dir = self.pproc_dir() + segment + '-cl'
                save_array(cls_dir, flowgen.classes)
                
                batch_arr.append((data, flowgen.classes, flowgen.class_indices))
            
            # Set the data.
            self.training_data = batch_arr[0][0]
            self.validation_data = batch_arr[1][0]
            
            # Classes are zero-indexed and represent a category in
            # numerical form. So if the classes are 'dog' and 'cat',
            # the possible class values will be 0 and 1.
            self.trn_classes = batch_arr[0][1]
            self.val_classes = batch_arr[1][1]
            
            # Labels are the one-hot encoded (i.e. categorical)
            # version of the classes. In other words, if there are
            # 5 classes and an element belongs to class 2,
            # its label will be [0,0,1,0,0] (index 1).
            self.training_labels = to_categorical(batch_arr[0][1])
            self.validation_labels = to_categorical(batch_arr[1][1])
            
            # Class indices are dictionaries of the form
            # {'category_name': 0, 'category_name_2: 1}. They
            # make the mapping between numerical class indices and
            # a human-readable category name. They are (should be...)
            # the same for validation and training, so only load them
            # once, after sanity checking.
            self.class_indices = batch_arr[0][2]
            print('Done preprocessing.')
        else:
            print('Loading data...')
            # Load the pre-saved data using methods defined in utils.py. See
            # preprocessing branch for the meaning of the data.
            self.training_data = load_array(self.pproc_dir() + 'train-bc')
            self.validation_data = load_array(self.pproc_dir() + 'valid-bc')
            self.trn_classes = load_array(self.pproc_dir() + 'train-cl')
            self.val_classes = load_array(self.pproc_dir() + 'valid-cl')
            self.training_labels = to_categorical(self.trn_classes)
            self.validation_labels = to_categorical(self.val_classes)
            
            # To get the class indices, we create the generator. It's cheap to
            # run since it doesn't actually load all the data.
            flowgen = gen.flow_from_directory(
                self.train_dir(),
                target_size=target_size,
                shuffle=False,
                class_mode=None,
                batch_size=1)    
            self.class_indices = flowgen.class_indices
            print('Done loading.')
        
    def trn_data(self):
        if self.training_data is None:
            self.load_data_and_labels()
        return self.training_data
    
    def val_data(self):
        if self.validation_data is None:
            self.load_data_and_labels()
        return self.validation_data
    
    def trn_labels(self):
        if self.training_labels is None:
            self.load_data_and_labels()
        return self.training_labels
    
    def val_labels(self):
        if self.validation_labels is None:
            self.load_data_and_labels()
        return self.validation_labels
        
    def __str__(self):
        return ('Options:\n'
            '  Testing directory: {0}\n'
            '  Training directory: {1}\n'
            '  Validation directory: {2}\n'
            '  Preprocess directory: {3}'
                .format(self.test_dir(),
                        self.train_dir(),
                        self.valid_dir(),
                        self.pproc_dir()))


opts = NotebookData(data_dir=data_root, sample_mode=True, preprocess=False)
class_names = [
  'safe driving',
  'texting - right',
  'talking on the phone - right',
  'texting - left',
  'talking on the phone - left',
  'operating the radio',
  'drinking',
  'reaching behind',
  'hair and makeup',
  'talking to passenger',
]
print(opts)

def save_and_create_submission(model,opt,name,generator):
    iter_path = opt.results_dir()+ '/' + name
    if not os.path.isdir(iter_path):
        os.makedirs(iter_path)
    model.save_weights(iter_path + '/temp_custom.h5')

    # Save the results to usable files.
    filenames = None
    batches = generator.flow_from_directory(opt.test_dir(),
                                            target_size=(224,224),
                                            class_mode=None,
                                            shuffle=False,
                                            batch_size=opts.batch_size(128))
    preds = model.predict_generator(batches, batches.n,verbose=1)
    filenames = batches.filenames
    predictions_path = iter_path + '/temp_custom_preds.dat'
    save_array(predictions_path, preds)
    print('Saved predictions to: %s' % predictions_path)
    filenames_path = iter_path + '/temp_custom_filenames.dat'
    save_array(filenames_path, filenames)
    print('Saved filenames to: %s' % filenames_path)
    
    # Create the response file.
    file_column = [pth[8:] for pth in filenames]
    clipped_preds = np.clip(preds, 0.05, 0.95)
    preds_col = [','.join(['%.2f' % p for p in pred]) for pred in clipped_preds]
    entries = [','.join([f,p]) for f,p in zip(file_column, preds_col)]
    entries = np.array(entries)

    class_names = ['c%d' % i for i in range(10)]
    title_row = ','.join(['img'] + class_names)

    submission_file_name = iter_path + 'submission.csv'
    np.savetxt(submission_file_name,
                entries,
                fmt='%s',
                header=title_row,
                comments='')
    
    FileLink(submission_file_name)
    
# Define a fit method to save on time.
def fit_model(model, tbatches, vbatches, opt, batch_size=8, epochs=5):
    bsize = opt.batch_size(batch_size)
    tbatches.batch_size = bsize
    vbatches.batch_size = bsize
    model.fit_generator(tbatches,
                        steps_per_epoch=math.ceil(len(opt.trn_labels())/bsize),
                        epochs=epochs,
                        validation_data=vbatches,
                        validation_steps=math.ceil(len(opt.val_labels())/bsize))
    
def load_model(model, opts, iter_name):
    model_dir = opts.results_dir() + '/' + iter_name + '/temp_custom.h5'
    print(model_dir)
    model.load_weights(model_dir)
    print('loaded')

Data root: /Users/alexandros/Development/DL/data/
Options:
  Testing directory: /Users/alexandros/Development/DL/data/statefarm/sample/test/
  Training directory: /Users/alexandros/Development/DL/data/statefarm/sample/train/
  Validation directory: /Users/alexandros/Development/DL/data/statefarm/sample/valid/
  Preprocess directory: /Users/alexandros/Development/DL/data/statefarm/sample/preprocessed/


In [25]:
# We have a proper model; we can now create a new batch
# generator with the training data and fit it.
batch_size = opts.batch_size(8)
gen = image.ImageDataGenerator()
training_batches = gen.flow(opts.trn_data(),
                            opts.trn_labels(),
                            batch_size=batch_size,
                            shuffle=True)
validation_batches = gen.flow(opts.val_data(),
                              opts.val_labels(),
                              batch_size=batch_size,
                              shuffle=False) # False so we always measure
                                             # validation the same way. (?)

Loading data...
Found 160 images belonging to 10 classes.
Done loading.


In [26]:
# Load the default model.
vgg_mod = Vgg16()

# Now that the model is loaded, get rid of the top layer;
# we want one that has 10 classes, not 1000.
vgg_mod.model.layers.pop()
for layer in vgg_mod.model.layers:
    layer.trainable = False
vgg_mod.model.add(Dense(len(list(iter(opts.class_indices))),
                        activation='softmax'))
vgg_mod.model.layers[-1].trainable = True
    
# We also need to modify the classes of the model, since
# the original model had 1000.
classes = list(iter(opts.class_indices))
for c in classes:
    classes[opts.class_indices[c]] = c
    
# Finally, we are ready to compile and run.
vgg_mod.model.compile(optimizer=Adam(lr=0.001),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
print('Tuning model compiled. Starting training.')
fit_model(vgg_mod.model,
          training_batches,
          validation_batches,
          opts,
          batch_size=64,
          epochs=1)

# batch_size 32
# lr 0.0001
# Epoch 1/5 446s 708ms/step - loss: 2.2702 - acc: 0.3011 - val_loss: 2.2284 - val_acc: 0.4955
# Epoch 2/5 445s 706ms/step - loss: 2.2088 - acc: 0.4162 - val_loss: 2.1634 - val_acc: 0.5188
# Epoch 3/5 446s 709ms/step - loss: 2.1546 - acc: 0.4484 - val_loss: 2.1029 - val_acc: 0.5312
# Stopped at 3 because this was waaaay too slow to converge.

# batch_size 32
# lr 0.001
# Epoch 1/5 446s 708ms/step - loss: 2.1414 - acc: 0.3171 - val_loss: 1.9004 - val_acc: 0.4585
# Epoch 2/5 445s 706ms/step - loss: 1.8797 - acc: 0.3969 - val_loss: 1.6483 - val_acc: 0.4665
# Epoch 3/5 445s 706ms/step - loss: 1.7331 - acc: 0.4138 - val_loss: 1.4862 - val_acc: 0.4946
# Epoch 4/5 445s 706ms/step - loss: 1.6395 - acc: 0.4329 - val_loss: 1.3518 - val_acc: 0.5277
# Epoch 5/5 444s 705ms/step - loss: 1.5764 - acc: 0.4596 - val_loss: 1.2777 - val_acc: 0.5580

# batch_size 32
# lr 0.01
# INTERRUPTED -- very bad, far too stable loss around 2.30, and unmoved through first epoch.

# batch_size 64
# lr 0.0001
# Epoch 1/5 441s 1s/step - loss: 2.2819 - acc: 0.2531 - val_loss: 2.2575 - val_acc: 0.4281
# Epoch 2/5 440s 1s/step - loss: 2.2451 - acc: 0.3873 - val_loss: 2.2185 - val_acc: 0.5000
# Epoch 3/5 441s 1s/step - loss: 2.2092 - acc: 0.4650 - val_loss: 2.1770 - val_acc: 0.5665
# Epoch 4/5 441s 1s/step - loss: 2.1743 - acc: 0.5153 - val_loss: 2.1378 - val_acc: 0.5875
# Epoch 5/5 440s 1s/step - loss: 2.1394 - acc: 0.5436 - val_loss: 2.0995 - val_acc: 0.6071

# batch_size 64
# lr 0.001
# Epoch 1/5 441s 1s/step - loss: 2.1987 - acc: 0.3219 - val_loss: 2.0766 - val_acc: 0.4165
# Epoch 2/5 441s 1s/step - loss: 1.9613 - acc: 0.4455 - val_loss: 1.7595 - val_acc: 0.5384
# Epoch 3/5 440s 1s/step - loss: 1.7564 - acc: 0.4863 - val_loss: 1.5306 - val_acc: 0.5879
# Epoch 4/5 440s 1s/step - loss: 1.6126 - acc: 0.5067 - val_loss: 1.3435 - val_acc: 0.6045
# Epoch 5/5 439s 1s/step - loss: 1.5050 - acc: 0.5225 - val_loss: 1.2419 - val_acc: 0.6147

# batch_size 64
# lr 0.001
# Epoch 1/10 439s 1s/step - loss: 2.1936 - acc: 0.2911 - val_loss: 2.0421 - val_acc: 0.3812
# Epoch 2/10 437s 1s/step - loss: 1.9871 - acc: 0.3536 - val_loss: 1.8273 - val_acc: 0.3929
# Epoch 3/10 438s 1s/step - loss: 1.8525 - acc: 0.3635 - val_loss: 1.6724 - val_acc: 0.4085
# Epoch 4/10 437s 1s/step - loss: 1.7709 - acc: 0.3646 - val_loss: 1.5642 - val_acc: 0.4295
# Epoch 5/10 437s 1s/step - loss: 1.7274 - acc: 0.3704 - val_loss: 1.4987 - val_acc: 0.4299
# Epoch 6/10 438s 1s/step - loss: 1.6781 - acc: 0.3772 - val_loss: 1.4572 - val_acc: 0.4304
# Epoch 7/10 437s 1s/step - loss: 1.6556 - acc: 0.3819 - val_loss: 1.4098 - val_acc: 0.4357
# Epoch 8/10 438s 1s/step - loss: 1.6403 - acc: 0.3829 - val_loss: 1.3909 - val_acc: 0.4339
# Epoch 9/10 439s 1s/step - loss: 1.6133 - acc: 0.3878 - val_loss: 1.3683 - val_acc: 0.4375
# Epoch 10/10 438s 1s/step - loss: 1.6191 - acc: 0.3886 - val_loss: 1.3883 - val_acc: 0.4540

# batch_size 64 (try 2)
# lr 0.001

Tuning model compiled. Starting training.
Epoch 1/1
160/160 [==============================] - 144s 897ms/step - loss: 2.3117 - acc: 0.0938 - val_loss: 2.3034 - val_acc: 0.1000


In [27]:
# Now that we've finetuned, we want to make every layer trainable
# to see if it improves the results.
first_dense_idx = [index
                   for (index, layer)
                   in enumerate(vgg_mod.model.layers)
                   if type(layer) is Dense][0]
for layer in vgg_mod.model.layers[:first_dense_idx]:
    layer.trainable = True
    
vgg_mod.model.compile(optimizer=Adam(lr=0.0001), # Slower convergence rate; we are already close.
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
print('All-dense-layer model compiled. Starting training.')
fit_model(vgg_mod.model,
          training_batches,
          validation_batches,
          opts,
          epochs=1)

All-dense-layer model compiled. Starting training.
Epoch 1/1
160/160 [==============================] - 342s 2s/step - loss: 2.3037 - acc: 0.1000 - val_loss: 2.3034 - val_acc: 0.1000


In [28]:
print(opts.results_dir())
save_and_create_submission(vgg_mod.model, opts, "test", gen)

/Users/alexandros/Development/DL/data/statefarm/sample/results
Found 16 images belonging to 1 classes.
16/16 [==============================] - 11s 692ms/step
Saved predictions to: /Users/alexandros/Development/DL/data/statefarm/sample/results/test/temp_custom_preds.dat
Saved filenames to: /Users/alexandros/Development/DL/data/statefarm/sample/results/test/temp_custom_filenames.dat


In [35]:
def load_model(model, opts, iter_name):
    model_dir = opts.results_dir() + '/' + iter_name + '/temp_custom.h5'
    print(model_dir)
    model.load_weights(model_dir)
    print('loaded')

load_model(vgg_mod.model, opts, 'test')

/Users/alexandros/Development/DL/data/statefarm/sample/results/test/temp_custom.h5
loaded
